In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3" 

In [7]:
import torch
import torch.nn as nn
from baukit import TraceDict

# 간단한 모델 정의 (예제)
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 10)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

# 모델 및 입력 데이터 생성
model = SimpleModel()
input_tensor = torch.randn(1, 10)

# 더할 벡터 정의
modification_vectors = {
    'fc1': torch.randn(1, 10),
    'fc2': torch.randn(1, 10),
    'fc3': torch.randn(1, 10),
}

# 연산 결과를 저장할 딕셔너리
operation_results = {}

# TraceDict 사용
with TraceDict(model, ['fc1', 'fc2', 'fc3'], edit_output=True) as traces:
    for layer in modification_vectors.keys():
        def modify_output(output, layer_name=layer):  # layer_name을 명시적으로 캡처
            added_vector = modification_vectors[layer_name].to(output.device)  # 벡터를 출력과 같은 장치로 이동
            modified_output = output + added_vector  # 벡터 추가

            # 연산 결과 저장 (두 벡터의 차이)
            operation_results[layer_name] = (output - added_vector).detach().cpu()

            return modified_output

        # 후킹된 출력 수정
        traces[layer].modification_hooks.append(modify_output)

    # 모델 실행
    output = model(input_tensor)

# 결과 출력
print("Final Output:", output)
print("Stored Operation Results:")
for layer, value in operation_results.items():
    print(f"{layer} Difference:", value)

AttributeError: 'Trace' object has no attribute 'modification_hooks'